In [1]:
import math
from scipy.constants import Planck, alpha, e, epsilon_0, speed_of_light, m_p, m_n, c
from random import choices
import simpy as simp
import random as rm
import numpy as np

In [2]:
#function for caculating the electric dipole capture
def cap_E(l,R,v):
    M = 1
    h = (6.5821*(10**-16))/(10**6)
    Jl = (3*(R**l))/(l+3)
    ke = ((2*math.pi*v)/c)
    Qlm = ((3*(3.5*(10**-8)))**l)*e
    Qlmp = ((ke*h)/(M*c))*Qlm
    w = 2*math.pi*v
    E__amp = -((4*math.pi)/(math.factorial(math.factorial(2*l + 1))))*(((l+1)/l)**0.5)*(ke**(l+2))*Qlm
    sig_cap_lm = ((8*math.pi*(l+1))/(l*math.factorial(math.factorial(2*l + 1))**2))*((ke**(2*l + 1))/h)*(abs(Qlm+Qlmp))
    Ue = (c/(2*math.pi*(ke**2)))*(abs(E__amp)**2)
    return sig_cap_lm, Ue

#function for calculating the magnetic dipole caputure    
def cap_M(l,R,v):
    M = 1
    h = (6.5821*(10**-16))/(10**6)
    Jl = (3*(R**l))/(l+3)
    ke = ((2*math.pi*v)/c)
    Qlm = ((3*(3.5*(10**-8)))**l)*e
    Mlm = Qlm*(v/c)
    Mlmp = (math.sqrt((10*(h/(M*c*R))**2)*(abs(Qlm)**2)))+Mlm
    w = 2*math.pi*v
    M__amp = +((4*math.pi)/(math.factorial(math.factorial(2*l + 1))))*(((l+1)/l)**0.5)*(ke**(l+2))*Qlm
    sig_cap_lm = ((8*math.pi*(l+1))/(l*math.factorial(math.factorial(2*l + 1))**2))*((ke**(2*l + 1))/h)*(abs(Mlm+Mlmp))
    Um = (c/(2*math.pi*(ke**2)))*(abs(M__amp)**2)
    return sig_cap_lm, Um

In [17]:
counts = 5000
Type = 'hw+D'
def neutron_collisions(env,counts,Type):
    M = 1
    E = 0
    Me = 0
    scat = 0
    testNum = 0
    l = 1
    A = 2
    R = (A**(1/3))*(10**-13)
    h = (6.5821*(10**-16))/(10**6)
    #for the formation of a duetron by neutron, proton capture
    if Type=='n+p':
        for i in range(counts+1):
            print('Test %d' % testNum)
            #neutron energy is randomly chosen from the range 8.5*10^10 and 10^11
            e = rm.randint(7*(10**18),1*(10**19))
            yield env.timeout(1)
            k = ((M*e)/(h**2))**(1/2)
            v = (2*h*k)/M
            cap__E,UE = cap_E(l,R,v)
            cap__M,UM = cap_M(l,R,v)
            react = ['cap E','cap M','scat']
            cross = [cap__E/(2*l +1),cap__M/(2*l + 1),1-((cap__E/(2*l +1))+(cap__M/(2*l + 1)))]
            #here the program takes a random weighted choice of the reactions using the probabilities that they will happen
            win=choices(react, cross)
            print(e/9.223e+18,'Kilo Joule')
            print(win)
            print(cross)
            if win==['cap E']:
                E+=1
                print(UE,UE)
            if win==['scat']:
                scat+=1
            elif win==['cap M']:
                Me+=1
                print(UE,UE)
            print(E,Me,scat,'\n')
            testNum+=1
    #for photodisitegration of the deutron
    elif Type=='hw+D':
        E = 0
        Me = 0
        scat = 0
        for i in range(counts+1):
            #photon energy is randomly chosen from this range:
            e = rm.randint(10,40)/(1*(10**19))
            k = ((M*e)/(h**2))**(1/2)
            v = (2*h*k)/M
            ke = ((2*math.pi*v)/c)
            cap__E,UE = cap_E(l,R,v)
            cap__M,UM = cap_M(l,R,v)
            dis_E = cap__E*((k/ke)**2)
            dis_M = cap__M*((k/ke)**2)
            react = ['dis E','dis M','com scattering']
            cross = [dis_E/(2*l +1),dis_M/(2*l + 1),1-((dis_E/(2*l +1))+(dis_M/(2*l +1)))]
            #here the program takes a random weighted choice of the reactions using the probabilities that they will happen
            win=choices(react, cross)
            print(e*1e+6,'eV')
            print(win)
            print(cross)
            if win==['dis E']:
                E+=1
                print(UE,UM)
            elif win==['dis M']:
                Me+=1
                print(UE,UM)
            elif win==['com scattering']:
                scat+=1
            print(E,Me,scat,'\n')
env = simp.Environment()
env.process(neutron_collisions(env,counts,Type))
env.run(until=counts+2)

2.5000000000000003e-12 eV
['com scattering']
[0.3159286751744235, 2.4074599997054442e-17, 0.6840713248255765]
0 0 1 

1.4999999999999999e-12 eV
['com scattering']
[0.1468303797047897, 1.0490653819812366e-17, 0.8531696202952103]
0 0 2 

3.8e-12 eV
['dis E']
[0.5920446053521343, 4.802407837626952e-17, 0.4079553946478657]
3.6673198455530704e-112 3.6673198455530704e-112
1 0 2 

3.4e-12 eV
['dis E']
[0.5010688547432413, 3.993953156416941e-17, 0.49893114525675875]
2.9358876325895804e-112 2.9358876325895804e-112
2 0 2 

1.2e-12 eV
['com scattering']
[0.1050632672582449, 7.325246697952839e-18, 0.8949367327417551]
2 0 3 

4e-12 eV
['com scattering']
[0.63939468269483, 5.2296938120182263e-17, 0.36060531730517]
2 0 4 

3.1e-12 eV
['com scattering']
[0.43623607187039976, 3.4287372141124085e-17, 0.5637639281296002]
2 0 5 

2.9999999999999997e-12 eV
['com scattering']
[0.41529902869380986, 3.24831100310742e-17, 0.5847009713061901]
2 0 6 

2e-12 eV
['com scattering']
[0.22606015799406753, 1.672289780

1.7168339443170898e-112 1.7168339443170898e-112
63 0 131 

1.9e-12 eV
['com scattering']
[0.2093193776047038, 1.5384482795173995e-17, 0.7906806223952962]
63 0 132 

1.6e-12 eV
['com scattering']
[0.16175548168930473, 1.1643702050307509e-17, 0.8382445183106952]
63 0 133 

3.8e-12 eV
['com scattering']
[0.5920446053521343, 4.802407837626952e-17, 0.4079553946478657]
63 0 134 

2.8e-12 eV
['com scattering']
[0.3744691805324423, 2.899611353924436e-17, 0.6255308194675576]
63 0 135 

2.3e-12 eV
['com scattering']
[0.2787858887116661, 2.1004062562752003e-17, 0.7212141112883339]
63 0 136 

4e-12 eV
['com scattering']
[0.63939468269483, 5.2296938120182263e-17, 0.36060531730517]
63 0 137 

1e-12 eV
['com scattering']
[0.07992433533685375, 5.47072172098473e-18, 0.9200756646631463]
63 0 138 

2e-12 eV
['dis E']
[0.22606015799406753, 1.6722897801257884e-17, 0.7739398420059325]
1.0158780735604085e-112 1.0158780735604085e-112
64 0 138 

3.1e-12 eV
['com scattering']
[0.43623607187039976, 3.42873721411

120 0 263 

1.1e-12 eV
['com scattering']
[0.09220788509395152, 6.371564499356043e-18, 0.9077921149060485]
120 0 264 

1.4e-12 eV
['com scattering']
[0.1323948484499297, 9.385911881325567e-18, 0.8676051515500703]
120 0 265 

1.7e-12 eV
['com scattering']
[0.17715459251516152, 1.2844199016326209e-17, 0.8228454074848385]
120 0 266 

1.2e-12 eV
['com scattering']
[0.1050632672582449, 7.325246697952839e-18, 0.8949367327417551]
120 0 267 

1.2e-12 eV
['com scattering']
[0.1050632672582449, 7.325246697952839e-18, 0.8949367327417551]
120 0 268 

1.7e-12 eV
['com scattering']
[0.17715459251516152, 1.2844199016326209e-17, 0.8228454074848385]
120 0 269 

1e-12 eV
['dis E']
[0.07992433533685375, 5.47072172098473e-18, 0.9200756646631463]
2.5396951839010207e-113 2.5396951839010207e-113
121 0 269 

2.9999999999999997e-12 eV
['com scattering']
[0.41529902869380986, 3.24831100310742e-17, 0.5847009713061901]
121 0 270 

1.8e-12 eV
['dis E']
[0.19301354661926903, 1.4091364107556594e-17, 0.80698645338073

177 0 375 

2.9e-12 eV
['dis E']
[0.39470809208243257, 3.071924844850869e-17, 0.6052919079175674]
2.135883649660759e-112 2.135883649660759e-112
178 0 375 

3.2e-12 eV
['com scattering']
[0.4575135919864151, 3.6131724551390055e-17, 0.5424864080135848]
178 0 376 

1.8e-12 eV
['com scattering']
[0.19301354661926903, 1.4091364107556594e-17, 0.8069864533807309]
178 0 377 

2.9e-12 eV
['dis E']
[0.39470809208243257, 3.071924844850869e-17, 0.6052919079175674]
2.135883649660759e-112 2.135883649660759e-112
179 0 377 

1.9e-12 eV
['com scattering']
[0.2093193776047038, 1.5384482795173995e-17, 0.7906806223952962]
179 0 378 

1.3000000000000001e-12 eV
['com scattering']
[0.1184660919240022, 8.330424281822376e-18, 0.8815339080759977]
179 0 379 

2.4e-12 eV
['com scattering']
[0.2971637949276782, 2.251800499085585e-17, 0.7028362050723218]
179 0 380 

2.9999999999999997e-12 eV
['com scattering']
[0.41529902869380986, 3.24831100310742e-17, 0.5847009713061901]
179 0 381 

2.5000000000000003e-12 eV
['di

209 0 449 

2.6000000000000002e-12 eV
['dis E']
[0.3350727077605235, 2.5673416557680927e-17, 0.6649272922394764]
1.7168339443170898e-112 1.7168339443170898e-112
210 0 449 

1.3000000000000001e-12 eV
['com scattering']
[0.1184660919240022, 8.330424281822376e-18, 0.8815339080759977]
210 0 450 

2.5000000000000003e-12 eV
['com scattering']
[0.3159286751744235, 2.4074599997054442e-17, 0.6840713248255765]
210 0 451 

2.9999999999999997e-12 eV
['com scattering']
[0.41529902869380986, 3.24831100310742e-17, 0.5847009713061901]
210 0 452 

4e-12 eV
['com scattering']
[0.63939468269483, 5.2296938120182263e-17, 0.36060531730517]
210 0 453 

1.3000000000000001e-12 eV
['com scattering']
[0.1184660919240022, 8.330424281822376e-18, 0.8815339080759977]
210 0 454 

2.8e-12 eV
['com scattering']
[0.3744691805324423, 2.899611353924436e-17, 0.6255308194675576]
210 0 455 

1.8e-12 eV
['dis E']
[0.19301354661926903, 1.4091364107556594e-17, 0.8069864533807309]
8.228612395839311e-113 8.228612395839311e-113
21

...

1e-12 eV
['com scattering']
[0.07992433533685375, 5.47072172098473e-18, 0.9200756646631463]
1476 0 2817 

2.5000000000000003e-12 eV
['dis E']
[0.3159286751744235, 2.4074599997054442e-17, 0.6840713248255765]
1.587309489938138e-112 1.587309489938138e-112
1477 0 2817 

1.7e-12 eV
['dis E']
[0.17715459251516152, 1.2844199016326209e-17, 0.8228454074848385]
7.339719081473946e-113 7.339719081473946e-113
1478 0 2817 

3.7e-12 eV
['com scattering']
[0.5688288558489311, 4.594495005006448e-17, 0.43117114415106894]
1478 0 2818 

2.6000000000000002e-12 eV
['com scattering']
[0.3350727077605235, 2.5673416557680927e-17, 0.6649272922394764]
1478 0 2819 

3.7e-12 eV
['dis E']
[0.5688288558489311, 4.594495005006448e-17, 0.43117114415106894]
3.4768427067604965e-112 3.4768427067604965e-112
1479 0 2819 

1.4e-12 eV
['com scattering']
[0.1323948484499297, 9.385911881325567e-18, 0.8676051515500703]
1479 0 2820 

1.7e-12 eV
['com scattering']
[0.17715459251516152, 1.2844199016326209e-17, 0.8228454074848385]
1

1531 0 2935 

2.5000000000000003e-12 eV
['com scattering']
[0.3159286751744235, 2.4074599997054442e-17, 0.6840713248255765]
1531 0 2936 

3.3e-12 eV
['com scattering']
[0.47912622552359113, 3.8015871698777444e-17, 0.5208737744764088]
1531 0 2937 

3.4e-12 eV
['dis E']
[0.5010688547432413, 3.993953156416941e-17, 0.49893114525675875]
2.9358876325895804e-112 2.9358876325895804e-112
1532 0 2937 

1.6e-12 eV
['dis E']
[0.16175548168930473, 1.1643702050307509e-17, 0.8382445183106952]
6.501619670786607e-113 6.501619670786607e-113
1533 0 2937 

1.1e-12 eV
['com scattering']
[0.09220788509395152, 6.371564499356043e-18, 0.9077921149060485]
1533 0 2938 

3.8e-12 eV
['com scattering']
[0.5920446053521343, 4.802407837626952e-17, 0.4079553946478657]
1533 0 2939 

3.1e-12 eV
['com scattering']
[0.43623607187039976, 3.4287372141124085e-17, 0.5637639281296002]
1533 0 2940 

1.9e-12 eV
['com scattering']
[0.2093193776047038, 1.5384482795173995e-17, 0.7906806223952962]
1533 0 2941 

3.6e-12 eV
['dis E']


1581 0 3049 

3.3e-12 eV
['com scattering']
[0.47912622552359113, 3.8015871698777444e-17, 0.5208737744764088]
1581 0 3050 

1.3000000000000001e-12 eV
['com scattering']
[0.1184660919240022, 8.330424281822376e-18, 0.8815339080759977]
1581 0 3051 

1.9e-12 eV
['com scattering']
[0.2093193776047038, 1.5384482795173995e-17, 0.7906806223952962]
1581 0 3052 

1.2e-12 eV
['com scattering']
[0.1050632672582449, 7.325246697952839e-18, 0.8949367327417551]
1581 0 3053 

3.4e-12 eV
['dis E']
[0.5010688547432413, 3.993953156416941e-17, 0.49893114525675875]
2.9358876325895804e-112 2.9358876325895804e-112
1582 0 3053 

4e-12 eV
['dis E']
[0.63939468269483, 5.2296938120182263e-17, 0.36060531730517]
4.063512294241633e-112 4.063512294241633e-112
1583 0 3053 

1.8e-12 eV
['com scattering']
[0.19301354661926903, 1.4091364107556594e-17, 0.8069864533807309]
1583 0 3054 

1.2e-12 eV
['com scattering']
[0.1050632672582449, 7.325246697952839e-18, 0.8949367327417551]
1583 0 3055 

3.9e-12 eV
['dis E']
[0.615567

2.6000000000000002e-12 eV
['com scattering']
[0.3350727077605235, 2.5673416557680927e-17, 0.6649272922394764]
1627 0 3178 

4e-12 eV
['dis E']
[0.63939468269483, 5.2296938120182263e-17, 0.36060531730517]
4.063512294241633e-112 4.063512294241633e-112
1628 0 3178 

2.9e-12 eV
['dis E']
[0.39470809208243257, 3.071924844850869e-17, 0.6052919079175674]
2.135883649660759e-112 2.135883649660759e-112
1629 0 3178 

3.4e-12 eV
['dis E']
[0.5010688547432413, 3.993953156416941e-17, 0.49893114525675875]
2.9358876325895804e-112 2.9358876325895804e-112
1630 0 3178 

2.3e-12 eV
['com scattering']
[0.2787858887116661, 2.1004062562752003e-17, 0.7212141112883339]
1630 0 3179 

3.3e-12 eV
['com scattering']
[0.47912622552359113, 3.8015871698777444e-17, 0.5208737744764088]
1630 0 3180 

1.4999999999999999e-12 eV
['com scattering']
[0.1468303797047897, 1.0490653819812366e-17, 0.8531696202952103]
1630 0 3181 

1e-12 eV
['com scattering']
[0.07992433533685375, 5.47072172098473e-18, 0.9200756646631463]
1630 0 

[0.47912622552359113, 3.8015871698777444e-17, 0.5208737744764088]
2.765728055268212e-112 2.765728055268212e-112
1689 0 3301 

2.4e-12 eV
['dis E']
[0.2971637949276782, 2.251800499085585e-17, 0.7028362050723218]
1.4628644259269884e-112 1.4628644259269884e-112
1690 0 3301 

3.7e-12 eV
['dis E']
[0.5688288558489311, 4.594495005006448e-17, 0.43117114415106894]
3.4768427067604965e-112 3.4768427067604965e-112
1691 0 3301 

2.6000000000000002e-12 eV
['dis E']
[0.3350727077605235, 2.5673416557680927e-17, 0.6649272922394764]
1.7168339443170898e-112 1.7168339443170898e-112
1692 0 3301 

2.1e-12 eV
['com scattering']
[0.2432248674565771, 1.81060018372879e-17, 0.7567751325434229]
1692 0 3302 

1.1e-12 eV
['com scattering']
[0.09220788509395152, 6.371564499356043e-18, 0.9077921149060485]
1692 0 3303 

2.9999999999999997e-12 eV
['com scattering']
[0.41529902869380986, 3.24831100310742e-17, 0.5847009713061901]
1692 0 3304 

1.6e-12 eV
['com scattering']
[0.16175548168930473, 1.1643702050307509e-17, 0